## Word Count
Counting the number of occurances of words in a text is one of the most popular first eercises when learning Map-Reduce Programming. It is the equivalent to `Hello World!` in regular programming.

We will do it two way, a simpler way where sorting is done after the RDD is collected, and a more sparky way, where the sorting is also done using an RDD.

In [2]:
# First, check that the text file is where we expect it to be
%ls ../Data/Moby-Dick.txt

../Data/Moby-Dick.txt


### Read the text file into an RDD
Note that, as execution is Lazy, this does not necessarily mean that actual reading of the file content has occured.

In [3]:
%%time
text_file = sc.textFile(u'../Data/Moby-Dick.txt')
type(text_file)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 805 ms


### Count the words
Next, we count the number of words that occured in the text. Again, this is only setting the plan.

In [5]:
from operator import add

In [9]:
%%time
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(add)
print type(counts)

<class 'pyspark.rdd.PipelinedRDD'>
CPU times: user 16 ms, sys: 12 ms, total: 28 ms
Wall time: 53.6 ms


### Have a look a the execution plan
Note that the earliest node in the dependency graph is the file `../../Data/Moby-Dick.txt`. It is possible that that even the first element in that file has not yet been read!

In [10]:
print counts.toDebugString()

(2) PythonRDD[18] at RDD at PythonRDD.scala:43 []
 |  MapPartitionsRDD[17] at mapPartitions at PythonRDD.scala:342 []
 |  ShuffledRDD[16] at partitionBy at NativeMethodAccessorImpl.java:-2 []
 +-(2) PairwiseRDD[15] at reduceByKey at <timed exec>:1 []
    |  PythonRDD[14] at reduceByKey at <timed exec>:1 []
    |  MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2 []
    |  ../Data/Moby-Dick.txt HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:-2 []


### Count!
Finally we count the number of times each word has occured.
Note that this cell, finaally, the Lazy execution model finally performs some actual work.

In [14]:
%%time
V_sz = counts.count()  
tf_total = counts.map(lambda (w,i): i).reduce(lambda x,y:x+y)  
print 'count=%f, sum=%f, mean=%f'%(V_sz, tf_total, float(tf_total)/V_sz)

count=33782.000000, sum=219480.000000, mean=6.496951
CPU times: user 4 ms, sys: 8 ms, total: 12 ms
Wall time: 258 ms


### Collect the `Sum` RDD into the driver node
This also takes significant work.

In [25]:
%%time
C = counts.collect()
print type(C)

<type 'list'>
CPU times: user 12 ms, sys: 8 ms, total: 20 ms
Wall time: 155 ms


### Sort 
Now that we have collected the Sum RDD into the driver node, we no longer rely on Spark. The following two cells
are simple python commands.

In [13]:
# sort using Python list 
C.sort(key=lambda x: x[1])
print 'most common words',C[-10:]
print 'Least common words',C[:10]

most common words [(u'I', 1724), (u'his', 2415), (u'that', 2693), (u'in', 3878), (u'', 4347), (u'to', 4510), (u'a', 4533), (u'and', 5951), (u'of', 6587), (u'the', 13766)]
Least common words [(u'funereal', 1), (u'unscientific', 1), (u'lime-stone,', 1), (u'shouted,', 1), (u'pitch-pot,', 1), (u'cod-liver', 1), (u'prices', 1), (u'prefix', 1), (u'boots."', 1), (u'slew.', 1)]


### Compute the mean number of occurances per word.

Without rely on spark

In [15]:
Count2 = len(C)
Sum2 = sum(i for w,i in C)
print 'count2=%f, sum2=%f, mean2=%f'%(Count2,Sum2,float(Sum2)/Count2)

count2=33782.000000, sum2=219480.000000, mean2=6.496951


## Word Count in Pure Spark
We now show how to perform word count, including sorting, using RDDs, returning to the driver node just the top 10 words.

In [16]:
%%time
RDD = text_file.flatMap(lambda x: x.split(' '))\
    .filter(lambda x: x!='')\
    .map(lambda word: (word,1))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 66 µs


In [17]:
%%time
RDD1 = RDD.reduceByKey(add)

CPU times: user 16 ms, sys: 4 ms, total: 20 ms
Wall time: 49.8 ms


In [18]:
%%time
RDD2 = RDD1.map(lambda (t, f): (f, t))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 31 µs


`sortByKey` takes significant amount of time.

In [21]:
%%time
RDD3 = RDD2.sortByKey(False)

CPU times: user 0 ns, sys: 24 ms, total: 24 ms
Wall time: 267 ms


In [22]:
print RDD3.toDebugString()

(2) PythonRDD[39] at RDD at PythonRDD.scala:43 []
 |  MapPartitionsRDD[38] at mapPartitions at PythonRDD.scala:342 []
 |  ShuffledRDD[37] at partitionBy at NativeMethodAccessorImpl.java:-2 []
 +-(2) PairwiseRDD[36] at sortByKey at <timed exec>:1 []
    |  PythonRDD[35] at sortByKey at <timed exec>:1 []
    |  MapPartitionsRDD[26] at mapPartitions at PythonRDD.scala:342 []
    |  ShuffledRDD[25] at partitionBy at NativeMethodAccessorImpl.java:-2 []
    +-(2) PairwiseRDD[24] at reduceByKey at <timed exec>:1 []
       |  PythonRDD[23] at reduceByKey at <timed exec>:1 []
       |  MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2 []
       |  ../Data/Moby-Dick.txt HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:-2 []


In [23]:
%%time
RDD3.take(10)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 210 ms


[(13766, u'the'),
 (6587, u'of'),
 (5951, u'and'),
 (4533, u'a'),
 (4510, u'to'),
 (3878, u'in'),
 (2693, u'that'),
 (2415, u'his'),
 (1724, u'I'),
 (1692, u'with')]